In [270]:
import numpy as np
import pandas as pd
from Util import LabelEncoderWithMissingValues
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

## 1. Import all the datasets

In [271]:
# test data
original_test = pd.read_csv(r"test_set_features.csv")

# train data
original_train = pd.read_csv(r"training_set_features.csv")

#target
training_labels = pd.read_csv(r"training_set_labels.csv")

# check whether rows are equal
print("train df => rows: %s, cols: %s" % (original_train.shape[0], original_train.shape[1]))
print("train labels df => rows: %s, cols: %s" % (training_labels.shape[0], training_labels.shape[1]))
print("test df => rows: %s, cols: %s" % (original_test.shape[0], original_test.shape[1]))

assert(original_train.shape[1] == original_test.shape[1])

train df => rows: 26707, cols: 36
train labels df => rows: 26707, cols: 3
test df => rows: 26708, cols: 36


## 2. Data cleaning

### 2.1 Check for duplicates

In [272]:
# check for duplicates
original_data_dup_count = np.sum(original_train.duplicated())
label_dup_count = np.sum(training_labels.duplicated())
original_test_dup_count = np.sum(original_test.duplicated())

print("duplicates in original training dataset: %s" % original_data_dup_count)
print("duplicates in label dataset: %s" % label_dup_count)
print("duplicates in original testing dataset: %s" % original_test_dup_count)

assert(original_data_dup_count == 0 and label_dup_count == 0 and original_test_dup_count == 0)

duplicates in original training dataset: 0
duplicates in label dataset: 0
duplicates in original testing dataset: 0


#### 2.2 Merging both train and test datasets

Before starting data cleaning, I need to merge them together for consistent results

In [273]:
original_train["type"] = "train"
original_test["type"] = "test"
original_data = pd.concat([original_train, original_test], ignore_index=True)

# the row count should be doubled and column count should be incremented by one
print("data => rows: %s, cols: %s" % (original_data.shape[0], original_data.shape[1]))

assert(original_data.shape[0] == original_train.shape[0] + original_test.shape[0])
assert(original_data.shape[1] == original_train.shape[1])

data => rows: 53415, cols: 37


Now `data` df has both datasets with different `Type` feature

Since there are no duplicates in the dataset, no need to drop duplicates

Before continuing, I'm setting `respondent_id` as index for the df

### 2.3 Setting `respodent_id` as index for the df

In [274]:
# set `respondent_id` as index
data = original_data.set_index("respondent_id")
training_labels = training_labels.set_index("respondent_id")

data.head(2)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,type
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,train
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,train


### 2.4 Encoding categorical data

This was not done in `attempt_1_170020C`. I am going with LabelEncoder to encode categorical features.
For that, I need to know what are the categorical features.

#### 2.4.1 Identifying categorical columns

I need to identify categorical columns and unique values for each column.

In [275]:
categorical_columns = np.where(data.dtypes == object)[0]

categorical_features = [data.columns[feature] for feature in categorical_columns]

for feature in categorical_features:
    print("%s:\t%s" % (feature, str(data[feature].unique())))

age_group:	['55 - 64 Years' '35 - 44 Years' '18 - 34 Years' '65+ Years'
 '45 - 54 Years']
education:	['< 12 Years' '12 Years' 'College Graduate' 'Some College' nan]
race:	['White' 'Black' 'Other or Multiple' 'Hispanic']
sex:	['Female' 'Male']
income_poverty:	['Below Poverty' '<= $75,000, Above Poverty' '> $75,000' nan]
marital_status:	['Not Married' 'Married' nan]
rent_or_own:	['Own' 'Rent' nan]
employment_status:	['Not in Labor Force' 'Employed' 'Unemployed' nan]
hhs_geo_region:	['oxchjgsf' 'bhuqouqj' 'qufhixun' 'lrircsnp' 'atmpeygn' 'lzgpxyit'
 'fpwskwrf' 'mlyzmhmf' 'dqpwygqj' 'kbazzjca']
census_msa:	['Non-MSA' 'MSA, Not Principle  City' 'MSA, Principle City']
employment_industry:	[nan 'pxcmvdjn' 'rucpziij' 'wxleyezf' 'saaquncn' 'xicduogh' 'ldnlellj'
 'wlfvacwt' 'nduyfdeo' 'fcxhlnwr' 'vjjrobsf' 'arjwrbjb' 'atmlpfrs'
 'msuufmds' 'xqicxuve' 'phxvnwax' 'dotnnunm' 'mfikgejo' 'cfqqtusy'
 'mcubkhph' 'haxffmxo' 'qnlwzans']
employment_occupation:	[nan 'xgwztkwe' 'xtkaffoo' 'emcorrxb' 'vlluhb

As you can see, `employment_industry`, `hhs_geo_region` and `employment_occupation` are scrambled for privacy protection.
However, I can still use those columns because, for my use case, knowing actual occupation or industry is not necessary.

Now I can apply LabelEncoding to those columns, because `NaN` values are depicted as `NaN`

#### 2.4.2 Applying Label Encoding to categorical columns

The problem with Native encoder is that it also encodes `NaN` values.
I have to make sure `Nan` values are unaffected as I'll be sanitizing them later

In [276]:
encoder, data = LabelEncoderWithMissingValues().categorical_to_numeric(data, ignored=['type'])

assert(data.shape[1] == original_data.shape[1] - 1)

data.tail(2)

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,type
respondent_id,,,,,,,,,,,,,,,,,,,,,
53413,3.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN,NaN,test
53414,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,3.0,2.0,0.0,0.0,NaN,2.0,test


As you can see `data` is now encoded with the exception of `type` column.
The type column will be later used to divide `data` into `train` and
`test`

### 2.5 Filling missing values

I want to use Nearest Neighbor Imputation to fill `NaN` values

#### 2.5.1 Normalizing data

KNN Imputer is distance based, and hence data should be normalized beforehand to avoid
bias.

In [277]:
temp = data.drop("type", axis=1)

scalar = MinMaxScaler()
temp = pd.DataFrame(scalar.fit_transform(temp), columns=temp.columns)

# note that `temp` has column `type` missing
temp.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,NaN,NaN
1,1.000000,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.5,0.111111,0.5,0.000000,0.0,0.00,0.000000
2,0.333333,0.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.5,0.0,0.0,0.5,0.222222,0.5,0.666667,0.0,0.05,0.045455
3,0.333333,0.5,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.333333,1.0,0.000000,0.0,NaN,NaN
4,0.666667,0.5,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,0.5,1.0,0.0,0.5,0.222222,0.5,0.333333,0.0,0.10,0.090909


#### 2.5.2 Utilizing KNN Imputer

In [278]:
# got the neighbour value `n_neighbors` from `attempt_1_170020C`
imputer = KNNImputer(n_neighbors=17)

temp = pd.DataFrame(imputer.fit_transform(temp), columns=temp.columns)

temp["type"] = data["type"]
data = temp

assert (not any(data.isna().any()))

data.tail()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,type
53410,0.333333,0.5,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.764706,0.411765,0.352941,0.888889,1.0,0.333333,0.333333,0.408824,0.304813,test
53411,1.000000,0.5,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,...,1.000000,1.000000,0.500000,0.222222,0.0,0.333333,1.000000,0.400000,0.136364,test
53412,0.000000,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.000000,0.000000,0.222222,0.5,0.333333,0.000000,0.252941,0.288770,test
53413,1.000000,0.5,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.000000,0.000000,0.000000,0.111111,0.5,0.333333,0.000000,0.447059,0.294118,test
53414,0.666667,0.5,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.000000,1.000000,0.500000,0.333333,1.0,0.000000,0.000000,0.320588,0.045455,test


### 2.6 Attempting to sort with K Neighbours Classifier

This is same classifier as `attempt_1_170020C`

#### 2.6.1 Define targets and train, test data

In [279]:
target_h1n1_vaccine = training_labels["h1n1_vaccine"].values
target_seasonal_vaccine = training_labels["seasonal_vaccine"].values

train_data = data[data.type.eq("train")].drop("type", axis=1)
test_data = data[data.type.eq("test")].drop("type", axis=1)


#### 2.6.2 Split train data into train and test

In [280]:
# split in 80:20 ratio
# because of two separate train and test scenarios, I'm using different names rather than use X_train, X_test...
p_train, p_test, q_train, q_test = train_test_split(train_data, target_h1n1_vaccine, test_size=0.2, random_state=1, stratify=target_h1n1_vaccine)
r_train, r_test, s_train, s_test = train_test_split(train_data, target_seasonal_vaccine, test_size=0.2, random_state=1, stratify=target_seasonal_vaccine)

#### 2.6.3 Use Grid Search CV

with K Neighbors Classifier separately for both h1n1 vaccine and seasonal vaccine

In [281]:
#### Use Grid Search to determine K value for each

param_grid = {
    "n_neighbors": [3, 5, 17, 21],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan"]
}
gscv = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=3, n_jobs=-1)

In [282]:
# for h1n1 vaccine

gscv.fit(p_train, q_train)

print(gscv.best_params_)
print(gscv.best_score_)
print(gscv.best_estimator_)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
{'metric': 'manhattan', 'n_neighbors': 21, 'weights': 'distance'}
0.8222793339869369
KNeighborsClassifier(metric='manhattan', n_neighbors=21, weights='distance')


In [283]:
# for seasonal vaccine
gscv.fit(p_train, q_train)

print(gscv.best_params_)
print(gscv.best_score_)
print(gscv.best_estimator_)



Fitting 3 folds for each of 16 candidates, totalling 48 fits
{'metric': 'manhattan', 'n_neighbors': 21, 'weights': 'distance'}
0.8222793339869369
KNeighborsClassifier(metric='manhattan', n_neighbors=21, weights='distance')
